## Using MatFlow to generate a stress-strain curve from DAMASK

## The workflow profile

### Initial options

We will use the example uniaxial simulation profile file located [here on GitHuB](https://github.com/LightForm-group/UoM-CSF-matflow/blob/master/workflows/DAMASK/uniaxial_sim.yml). Let's go through the different sections of the workflow file. First of all, we see this block:

```yaml
name: uniaxial_sim
stats: false
archive: dropbox
run_options:
  l: short
```

The `name` parameter is used when MatFlow generates a directory for the workflow; this directory will be the name, followed by a date and time stamp. Here, `stats` is set to `false`, meaning MatFlow will not run extra accounting jobs to record extra information like the memory usage, which is usually not needed, but might be useful when performing benchmarking. The `archive` key is set in this case to `dropbox`, which is the name of an archive location definition within my `~/.matflow/config.yml` file. Within the config file, the archive location is defined like this:

```yaml
archive_locations:
  dropbox:
    cloud_provider: dropbox
    path: /matflow/archived
```

In the above, the `path` key is defined relative to the root directory in my Dropbox account.

Finally, `run_options` specifies the default scheduler options that should be used by the tasks. Most of these options are passed directly to the scheduler, with the exception of `num_cores` and `alternate_scratch`.

### Tasks

Now let's look briefly at the tasks defined in this workflow. The first task is:

```yaml
  - name: generate_microstructure_seeds
    method: random
    software: damask
    base:
      grid_size: [16, 16, 16]
      num_grains: 100
    output_map_options:
      phase_label: Al  
```

This task generates seed positions for the microstructure; these will become the grain centres. The initial RVE discretisation is specified here as well using the `grid_size` option. Notice also the use of the `output_map_options` key. This is used to specify any additional meta information that should be associated with the task output, but that does not actually modify the execution of the task itself. Here, we are using it to tell label the generated microstructure seed positions as belonging to a phase labled by `Al`.

The second task is to generate the model geometry that will be simulated by DAMASK:
    
```yaml
  - name: generate_volume_element
    method: random_voronoi
    software: damask
    base:
      size: [1, 1, 1]
```

We specify that the volume element should be generated by using a random voronoi tessellation, in this case using the processing tools available in DAMASK.

The next task is simply to provide a visualisation of the volume element, in the form of a VTR file, which can be viewed directly within Paraview:
    
```yaml
  - name: visualise_volume_element
    method: VTK
    software: damask
```

After this, we need to generate a load case for the simulation, to tell DAMASK by how much to deform the volume element. This task is specified like this:

```yaml
  - name: generate_load_case
    method: uniaxial
    software: formable
    base:
      total_times: [100]
      num_increments: [300]
      target_strain_rates: [1.0e-3]
      directions: [z]
```

Here, we specify that the loading should be uniaxial in the model z-direction. We also specify for how long loading should continue, and over how many increments DAMASK should partition the load case. Notice that the loading keys specified here are lists, rather than single numbers. This is to support multiple sequential load cases within the same simulation. For instance, we could load first at a strain rate of 1e-3, and then at a strain rate of 1e-4. Alternatively, we might want to increase the number of increments per unit of strain in the elastic-plastic transition region, to make sure we sample the curvature sufficiently well.

Finally, we have the task to perform the actual simulation using DAMASK:

```yaml
  - name: simulate_volume_element_loading
    method: CP_FFT
    software: DAMASK
    run_options:
      num_cores: 8
      alternate_scratch: /mnt/eps01-rds/Fonseca-Lightform/mbdxqap3/temp-DAMASK
    output_map_options:
      operations:
        - name: add_Cauchy
          args: {P: P, F: F}
          opts: {add_Mises: true}
        - name: add_strain_tensor
          args: {F: F, t: V, m: 0}
          opts: {add_Mises: true}          
        - name: add_strain_tensor
          args: {F: Fp, t: V, m: 0}
          opts: {add_Mises: true}
      incremental_data:
        - name: vol_avg_equivalent_stress
          path: constituent/1_Al/generic/sigma_vM
          transforms: [mean_along_axes: 1]
        - name: vol_avg_equivalent_strain
          path: constituent/1_Al/generic/epsilon_V^0(F)_vM
          transforms: [mean_along_axes: 1]
        - name: vol_avg_equivalent_plastic_strain
          path: constituent/1_Al/generic/epsilon_V^0(Fp)_vM
          transforms: [mean_along_axes: 1]
          increments: 10 # Data should be extracted every 10th increment.
    base:
      homogenization_labels: [SX]
      homogenization_schemes:
        SX:
          mech: none
      phases:
        Al:
          outputs: [F, P, Fp]
          tau0_slip: 22e6
          tausat_slip: 129e6
          h0_slipslip: 573e6
          elasticity: hooke
          plasticity: phenopowerlaw
          lattice_structure: fcc
          Nslip: 12
          Ntwin: 0
          c11: 106.75e9
          c12: 60.41e9
          c44: 28.34e9
          gdot0_slip: 0.001
          n_slip: 20
          a_slip: 2.25
          interaction_slipslip: 1 1 1.4 1.4 1.4 1.4
          atol_resistance: 1
```

There are a few things to note here:
- We specify `run_options` here that differ from the default options we specified for all other tasks. This is because the simulation itself is, relatively, computationally expensive, and so requires more CPU cores to complete. Here we specify 8 cores. Another run option is the `alternate_scratch` key, which tells MatFlow to execute this task within a different directory to the MatFlow project directory (which is best to keep on scratch). This is used here because DAMASK cannot run on the CSF's scratch filesystem; yet it is best-practice to run as much computational work as possible on scratch; by specifying the `alternate_scratch` key (here, pointing to a location on our group RDS space), the DAMASK input files will be transparently copied over to the alternate location, and once the simulation is complete, the output files will be copied back to the project directory to allow the workflow to continue.
- The `output_map_options` tell MatFlow which data from the DAMASK simulation should be saved into the workflow
- There must be a phase listed in the `phases` key that matches the `phase_label` defined in the first task: `generate_microstructure_seeds`.

## Plotting a stress-strain curve from the results

In [1]:
from matflow import load_workflow
from plotly import graph_objects

In [2]:
path = r'/path/to/matflow/project/directory'
workflow = load_workflow(path)

Loading matflow config from C:\Users\adamj\.matflow\config.yml
Loading task schemas from 1 file(s)...OK!
Loading software definitions from 1 file(s)...OK!


In [3]:
workflow.tasks

[Task(name='generate_microstructure_seeds', method='random', software='damask'),
 Task(name='generate_volume_element', method='random_voronoi', software='damask'),
 Task(name='generate_load_case', method='uniaxial', software='formable'),
 Task(name='visualise_volume_element', method='VTK', software='damask'),
 Task(name='simulate_volume_element_loading', method='CP_FFT', software='damask')]

In [4]:
vol_elem_resp = workflow.tasks[4].elements[0].outputs.volume_element_response
strain = vol_elem_resp['vol_avg_equivalent_strain']['data']
stress = vol_elem_resp['vol_avg_equivalent_stress']['data']

In [5]:
graph_objects.FigureWidget(
    data=[{
            'x': strain,
            'y': stress,
    }],
    layout={'width': 550, 'xaxis_title': 'Eq. strain', 'yaxis_title': 'Eq. stress'}
)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'b61604d4-8b30-48b6-8817-0257022c926f',
 …